# Machine Translation mit Encoder-Decoder Modell

In diesem Notebook möchten wir uns mit der Königsdisziplin des Natural Language Processings beschäftigen, der maschinellen Übersetzung. Vermutlich gibt es keine NLP-Anwendung, die einerseits vielen bekannt ist, anderseits durch Deep Learning und neuronalen Netzen einen solchen Aufschwung bekommen hat.

Vor dem Siegeszug der neuronalen Netze, wurde maschinelle Übersetzung deshalb als schwierig angesehen, weil diese Aufgabe alle Teilaspekte von Sprache beinhaltet. Neben grammatikalischer Korrektheit, sollen maschinell übersetzte Texte den Sinn des Originaltextes wiedergeben und darüberhinaus auch den Subtext, wie Ironie, Witz, erfassen. Ob letzteres einfach gelingt, sei hier mal dahingestellt.

Deshalb möchten wir in diesem Notebook ein ML Modell aufbauen, dass englische Sätze auf Deutsch übersetzt.

## 0. GPU-Nutzung
Diesmal ist es empfehlenswert fürs Training eine GPU zu nutzen. Bevor Ihr das Training startet, könnt ihr mit dem folgenden Code überprüfen ob in Colab eine GPU registriert ist. Bitte nutzt die GPU erst, wenn Ihr mit der Implementierung fertig seid, um Ressourcen zu sparen.

In [0]:
%tensorflow_version 2.x
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

## 1. Daten
Im Vergleich zu den bisherigen Notebooks sind die Daten für Machine Translation relativ unspannend. Es handelt sich um Englisch-Deutsche-Satzpaare. Daher möchten wir uns hiermit nicht allzu lange aufhalten. Ladet euch wie gewohnt die Daten aus Google Drive. (Falls das nicht klappt: [hier](www.manythings.org/anki/deu-eng.zip) findet ihr die Daten).


Danach wollen wir die Daten einlesen. Das Format ist einfach: pro Zeile steht ein Satzpaar getrennt von einem Tab.

In [0]:
!pip install googledrivedownloader

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1ECd2plUitjNkU-xdo9bWCNRcscTgOoVu',
                                    dest_path='./download/deu-eng.zip',
                                    unzip=True,
                                    overwrite=True)

In [0]:
with open('./download/deu.txt', 'r') as f:
    lines = f.readlines()

In [0]:
pairs = [tuple(l.split("\t")) for l in lines]

In [0]:
pairs_cleaned = [(e, g.strip()) for e, g, _ in pairs]

Nutzt den `TweetTokenizer` von NLTK, um die englischen und deutschen Sätze in Tokens umzuwandeln.
Damit unser Modell weiß wo Anfang und Ende der deutschen Sätze sind, müssen wir noch ein spezielles Start- (🏳️) und Endsymbol (🏴) einfügen:

In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = #TODO

In [0]:
pairs_cleaned_with_marker = [... for e, g in pairs_cleaned] #TODO

Sammelt nun alle Tokens, die in den englischen bzw. den deutschen Sätzen vorkommen, in jeweils einem Set ab.

In [0]:
english_tokens = set()
german_tokens = set()

#TODO

Um unser Modell später mit passenden Eingabedaten füttern zu können, müssen wir nun noch die jeweiligen Vokabularindices für den Encoder und den Decoder bestimmen. Wie gewohnt nehmen wir dafür Dictionaries, die jeweils die Tokens auf den richtigen Index abbilden. 
Damit wir später unsere Batches padden können, fügen wir noch ein `PADDING`-Token ein, das den Index `0` hat.

In [0]:
english_token_index = #TODO
german_token_index = #TODO

In [0]:
assert english_token_index["PADDING"] == 0
assert german_token_index["PADDING"] == 0

Speichert die Längen des Vokabulars in den Variablen  `english_tokens_len` und `german_tokens_len` ab.

In [0]:
english_tokens_len = #TODO
german_tokens_len = #TODO
print(f"Englisch: {english_tokens_len}")
print(f"Deutsch: {german_tokens_len}")

Um die Verarbeitung der Sätze zu vereinfachen, speichern wir nun keine Wörter mehr, sondern Wortindices. Im nächsten Schritt sollt ihr deshalb `pairs_cleaned_with_marker` in das Format z.B. `[([1,4,6,7,...],[7,1,2,8,...]),([1,5,9,7,...],[2,7,9,8,...]), ...]` umwandeln. (Tokens --> Zahlen)

In [0]:
sentences_idx=[... for e,g in pairs_cleaned_with_marker] #TODO

### 1.1 Data Generator

Wir haben nun alle Dimensionen bestimmt, um die Trainingsdaten zu definieren. Dafür bauen wir uns mal wieder einen Data Generator. Da wir relativ viele Daten haben, müssen wir die Daten pro Batch generieren und in das Model laden. Da wir diesmal ein relativ kompliziertes Machine Learning Model bauen, haben wir für euch den Generator schon fast ganz vorimplementiert. 
Ihr müsst nur noch die methode `__data_generation()` implementieren.

Wir nutzen später `model.fit_generator()`, in der die Funktion `__getitem()__` aufgerufen wird, um einen Batch in das Modell zu laden.

Wir erzeugen uns zuerst Daten, um unseren Encoder und Decoder zu trainieren. Später werden wir die Architektur umbauen, um unsere eigentliche Predictions zu machen.
Um unser Model zu trainieren bekommt es sowohl englische als auch deutsche Satzpaare als Input:

```
encoder_input=[["hi", ",", "my","name","is", "tom"]]
deoder_input=[["🏳️", "hallo", ",", "mein","name","ist", "tom", "🏴"]]
```
Die Ausgabe unseres Models soll wieder der Deutsche Satz sein, aber diesmal bauen wir ein offset von 1 ein, da wir immer gegeben den Kontext das nächste Wort vorhersagen möchten:

```
decoder_outputs=[["hallo", ",", "mein","name","ist", "tom", "🏴"]]
```


---


Da unser Model natürlich keine Wörter, sondern Zahlen nimmt soll die Rückgabe eines Batches in folgendem Format erfolgen:
  
  `([encoder_inputs, decoder_inputs], decoder_outputs)`

* `encoder_inputs` ist ein Batch unserer *englischen* Sätze im Wortindex-Format (englisches Vokabular),
z.B. ```[[idx_1,idx_2][idx_2,idx_3],[...]]```

* `decoder_inputs` ist ein Batch unserer *deutschen* Sätze im Wortindex-Format (deutsches Vokabular),
z.B. ```[[idx_1,idx_2][idx_2,idx_3],[...]]```

Um eine Sequenz zu übersetzen, wollen wir später die  `decoder_outputs` vorhersagen. Hierbei handelt es sich um eine Klassifikation auf dem gesamten deutschen Vokabular. 

* Wir bauen hier noch ein offset von `1` ein und die `decoder_outputs` sehen dann für einen Batch bspw. folgendermaßen aus:
```[[[0,0,0],[1,0,0],[0,1,0]],[[0,0,0],[0,1,0],[0,0,1]],[...]]```

**Padding**: Wir hatten zuvor das `PADDING`-Token angesprochen. Ein Problem, dass man häufig bei Texten hat, ist, dass diese unterschiedlich lang sind. Da unser Netz nur mit Tensoren umgehen kann und es im Batch immer eine gleiche Länge erwartet, müssen wir alle zu kurzen Sätze  "padden". Wir füllen deshalb alle Sätze, die kürzer sind als der längste Satz mit `0` (bzw. mit dem `PADDING`-Token) auf. 

> Wichtig: Die `encoder_inputs` werden links und die `decoder_inputs` rechts gepadded.

In [0]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, sentences_idx, german_tokens_len ,batch_size=64, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.sentences_idx = sentences_idx
        self.german_tokens_len=german_tokens_len
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.sentences_idx) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of Indexes for batch
        list_IDXs_batch = [self.sentences_idx[k] for k in indexes]

        # Generate data
        encoder_inputs, decoder_inputs, decoder_outputs=self.__data_generation(list_IDXs_batch)

        return ([encoder_inputs, decoder_inputs], decoder_outputs)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.sentences_idx))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDXs_batch):
        '''
        Generates data containing batch_size samples.
        Padds the sentences in a batch to the longest sentence in a batch.
        encoder_input are left padded and decoder_input are right padded.
        The decoder output is a List of one-hot-encoded Vectors for each Sentence.
        list_IDXs_batch --> [([english_idxs],[german_idxs]),([english_idxs],[german_idxs]),...]
        '''

        #TODO


        return encoder_input, decoder_input, decoder_output

Nutzt nun den fertigen Generator und splittet `sentences_idx` in einen Train- und Validation-Datensatz.

In [0]:
import numpy as np
# TODO

train, val = # TODO
train_samples_len= # TODO

## 2. Sequence-to-sequence Modelle

 
Für unser Machine Translation Model bauen wir in diesem Teil ein Seq2Seq-Modell auf, das aus einem Encoder und einem Decoder besteht. Diese basieren jeweils wieder auf `LSTM`-Zellen, um Abhängigkeiten in den Sequenzen zu lernen. 

Hierfür nutzen wir die Functional API von Keras. Im Gegensatz zu der Sequence-API, definiert man hier ein Modell, sodass eine Layer mit der jeweiligen Vorgängerlayer aufgerufen wird und ein Tensor zurückgegeben wird, z.B:

```
a = Input(...)
b = Dense(...)
t = b(a)
```



Nutzt für das LSTM und das spätere Training folgende Parameter:

In [0]:
embedding_len = 50      # Length of the vector that we will be returned from the embedding layer
latent_dim    = 256     # Hidden layers dimension 
batch_size    = 64      # Batch size
epochs        = 50      # Number of epochs

Bevor wir mit dem Encoder starten, erstellt Euch mit dem zuvor gebauten DataGenerator und den gesplitteten Daten je einen Generator fürs Trainieren und einen fürs Validieren.

In [0]:
generator_train = DataGenerator(#TODO)
generator_val = DataGenerator(#TODO)

### 2.1 Encoder

Auch das komplexeste Modell fängt mit einer Eingabe ein. 

Definiert einen Input-Layer, der als Eingabe englische Sätze beliebiger Länge als Vektoren akzeptiert. Eine Komponente im Vektor repräsentiert je ein Wort. 

* Da wir später auch kleinere Batchgrößen (z.B. nur einen Satz) vorhersagen möchten, setzt die Batch-Size des Input-Layers auf `None`.
* Da wir jeweils nur in einem Batch padden, um effizienter zu trainieren, kommt es vor, dass die Sätze in unterschiedlichen Batches auf unterschiedliche Längen gepadded werden. Damit ist es nicht möglich eine eindeutige Satzlänge vorzugeben. Keras unterstützt das, indem wir die 2. Dimension des Input-`shapes` einfach leer lassen.

Der shape im Input sollte also so aussehen: `shape=(None,)`


In [0]:
from keras.layers import Input
encoder_inputs = #TODO

Erstellt euch nun einen Embedding Layer für das englische Vokabular.

In [0]:
from keras.layers import Embedding
encoder_embedding_layer = #TODO
encoder_embeddings = #TODO

Als nächstes definiert einen LSTM-Layer, der 256 hidden units hat. Die Ausgabe des Layers soll später an den Decoder 'gefüttert' werden. Seht in der Dokumentation nach, wie man auf den Thought-Vektor zugreifen kann (https://keras.io/layers/recurrent/)

In [0]:
from keras.layers import LSTM 
encoder_lstm_layer = #TODO

Verbindet nun den `encoder_embeddings`-Layer mit dem `encoder_lstm_layer`.

In [0]:
_, encoder_state_h, encoder_state_c = #TODO

Der Encoder ist somit fertig implementiert.

### 2.2 Decoder

Auch der Decoder fängt mit einem Input-Layer an. Definiert einen Input-Layer analog zum Encoder.

In [0]:
decoder_inputs = # TODO

Erstellt Euch nun einen Embedding Layer für das deutsche Vokabular.

In [0]:
decoder_embedding_layer = # TODO
decoder_embeddings = # TODO

Wie auch im Encoder, ist das Herzstück ein LSTM-Layer. Definiert einen LSTM-Layer mit 256 hidden units. Seht in der Dokumentation nach, wie man neben den Thought-Vektoren, auch die komplette Ausgabe-Sequence zurückbekommt.

In [0]:
decoder_lstm_layer = #TODO

Verbindet nun den `decoder_embeddings`-Layer mit dem `decoder_lstm_layer`. An welcher der drei Ausgaben sind wir hier interessiert? Außerdem müssen wir hier, den Thought-Vektor des Encoders mit übergeben, dies geschieht über den `initial_state`-Parameter

In [0]:
decoder_output_states,_,_= # TODO

Den Abschluss unseres Modells bildet ein Dense-Layer, der gleich viele `hidden_units` wie der Embedding-Layer des Decoders hat, da er wieder auf das deutsche Vokabular abbildet. Als Aktivierungsfunktion nehmen wir die Softmax-Funktion.

In [0]:
from keras.layers import Dense

decoder_dense_layer = # TODO

Verbindet diesen Layer mit dem bisherigen Decoder.

In [0]:
decoder_outputs= # TODO

### 2.3 Encoder-Decoder

Wir haben nun alle Elemente für ein Sequence-To-Sequence-Modell zusammen. Im letzten Schritt bauen wir das ganze Modell zusammen.

In [0]:
from keras import Model

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_outputs])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
model.summary()

Mit der folgender Code-Zelle könnt ihr euch die Modelstruktur genauer anzeigen lassen:

In [0]:
from keras.utils import plot_model
plot_model(model,show_shapes=True, show_layer_names=True)

Nun können wir unser Encoder-Decoder Modell trainieren. Definiert wie gewohnt eure Callbacks (bspw. `EarlyStopping` und `ModelCheckpoint`).

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping= # TODO
model_checkpoint = # TODO
callbacks = [model_checkpoint, early_stopping]

Voilá, nutzt nun `model.fit_generator()` um das Modell zu trainieren.

In [0]:
model.fit_generator(
    generator=generator_train, 
    validation_data=generator_val, 
    callbacks=callbacks, 
    epochs=epochs
)

## 3. Übersetzung
Jetzt kommen wir zum spannenden Teil der Aufgabe. Aktuell ist unser Modell für das Training noch so aufgebaut, dass der deutsche Satz als Eingabe für den Decoder benötigt wird.  In diesem Schritt bauen wir unser Model so um, dass es nur mit dem englischen Satz den deutschen Satz generieren (aká übersetzen) kann.

Zuerst ist allerdings ein kleiner Zwischenschritt vonnöten. Damit wir aus unseren Indizes wieder Texte bekommen, müssen wir noch das Dictionary `reverse_german_token_index` befüllen, das die "Umkehrung" von `german_token_index` ist. Befüllt für Testzwecke analog auch `reverse_english_token_index`.

In [0]:
reverse_german_token_index = # TODO
reverse_english_token_index = # TODO

### 3.1 Inferenzmodell

Definiert ein `Model`, das einen Eingabetext enkodiert. Als Eingabe hat das `Model` den bisherigen `encoder_input`-Layer und die Thought-Vektoren des Encoders als Output. 


In [0]:
encoder_model = Model() # TODO

In [0]:
encoder_model.summary()

Etwas komplizierter ist das Decoder-Modell. Als Input nimmt es zum einen die Thought-Vektoren des Encoders, zum anderen den `decoder_input`-Layer von oben.
Für den Docoder haben wir schon mal etwas vorgeben.

In [0]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]

Neben den beiden Thought-Vektoren des Encoders dient auch die dekodierte Sequenz als Eingabe. Deshalb müssen wir den Embedding-Layer des Decoders als Input übergeben. Definiert erneut einen Input-Layer und verknüpft diesen mit dem trainierten Embedding-Layer des Decoders. 

In [0]:
decoder_inputs_2 = #TODO

In [0]:
decoder_embedding_layer_2=#TODO

Setzt die beiden Input-Layer `decoder_states_inputs` als `initial_state` und den `decoder_embedding_layer_2`-Layer als Input in den `decoder_lstm_layer` von oben ein.

In [0]:
decoder_outputs_2, decoder_state_h, decoder_state_c = #TODO
# Speichert die docder_states separat ab
decoder_states = #TODO

Setzt nun den `decoder_outputs_2`-Tensor in den `decoder_dense_layer` von oben ein.

In [0]:
decoder_final_outputs = #TODO

Nun können wir ein `Model` zusammenbauen, das uns als Decoder dient. Input ist hier der `decoder_inputs_2`-Layer, zusammen mit den beiden Tensoren die wir in `decoder_states_inputs` gespeichert haben. Als Output dient uns der `decoder_final_outputs`-Tensor und die beiden Decoder Output-Tensoren, die sich in `decoder_final_output` befinden. 

In [0]:
decoder_model = Model(
    [decoder_inputs_2] + decoder_states_inputs,
    [decoder_final_outputs] + decoder_states
)

In [0]:
plot_model(decoder_model,show_shapes=True, show_layer_names=True)

Nun können wir eine Funktion `translate` implementieren, die als Eingabe englische Vokabularindizes nimmt. Wir haben dies schon mal vorbereitet. Schaut Euch trotzdem die einzelnen Schritte in der Funktion an, um zu verstehen wie eine Übersetzung erzeugt wird.


In [0]:
def translate(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = german_token_index['🏳️']# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)# Sample a token
      
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_german_token_index[sampled_token_index]
        
        # or find stop character.
        if (sampled_token == '🏴' or
           len(decoded_sentence) > 52):
            break
        decoded_sentence += ' '+sampled_token# Exit condition: either hit max length
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index# Update states
        states_value = [h, c]
        
    return decoded_sentence


Nun können wir testen, wie gut unser Übersetzer funktioniert. Am besten ihr testet das Ganze mit ein paar Sätzen aus dem Validation Set. 
Wichtig: die Eingabe muss im Batch erfolgen!
Nutzt im Zweifel `np.expand_dims()`

In [0]:
english_text=#TODO
german_text=translate(english_text)
print(german_text)

## 4. Optionale Zusatzaufgabe
Ihr könnt in euren Embedding-Layern noch vortrainierte Word Embeddings nutzen. Dies sollte bessere Ergebnisse liefern und die Generalisierungsfähigkeiten des Models verbessern.